# Emotiv Epoc X

In [24]:
# %matplotlib widget # Inline interactive plots
%matplotlib qt 
# Windowed interactive plots

## Import libraries
import os
import mne
import sys
sys.path.append("..") # Adds higher directory to python modules path.
import numpy as np
import scipy.signal as signal
import matplotlib.pyplot as plt
from Functions import artifact_removal_tools as art

#%% Import and organize
# - Import data
# full_path = 'C:\\Users\\danie\\OneDrive - University of Calgary\\PDF\\Proyects\\SSVEP_ART\\Data\\Emotiv Epoc X\\LF\\HeadRolls_LF_Mar23_EPOCX_126787_2021.03.23T17.22.54.06.00.edf'
# full_path = 'C:\\Users\\danie\\OneDrive - University of Calgary\\PDF\\Proyects\\SSVEP_ART\\Data\\Emotiv Epoc X\\LF\\SSVEP_10Hz_LF_Mar23_EPOCX_126787_2021.03.23T17.18.27.06.00.edf'
# full_path = 'C:\\Users\\danie\\OneDrive - University of Calgary\\PDF\\Proyects\\SSVEP_ART\\Data\\Emotiv Epoc X\\OKB\\SSVEP_10Hz_OKB_Mar22_EPOCX_126582_2021.03.22T17.42.37.06.00.edf'
full_path = 'C:\\Users\\danie\\OneDrive - University of Calgary\\PDF\\Proyects\\SSVEP_ART\\Data\\Emotiv Epoc X\\OKB\\SSVEP_17Hz_OKB_Mar22_EPOCX_126582_2021.03.22T17.45.11.06.00.edf'
edf_data = mne.io.read_raw_edf(full_path)

# - Organize data
srate = edf_data.info['sfreq']          # Sampling frequency [Hz]
ch_names = edf_data.info['ch_names']    # Channel names [list]
raw_data = edf_data.get_data(range(4,18))          # Raw data [channels x samples] [V]

#%% Notebook settings
plot_edf = True # Boolean to plot raw data


Extracting EDF parameters from C:\Users\danie\OneDrive - University of Calgary\PDF\Proyects\SSVEP_ART\Data\Emotiv Epoc X\OKB\SSVEP_17Hz_OKB_Mar22_EPOCX_126582_2021.03.22T17.45.11.06.00.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


## Plot EDF data

In [25]:
if plot_edf:
    edf_data.plot()

## Separate channels

In [26]:

time_trim = [45, 55]  # Time start and end to trim EEG [sec]
np.diff(time_trim)
time_vect = np.linspace(time_trim[0], time_trim[1], int(np.diff(time_trim)*srate))
raw_trim = raw_data[:,int(time_trim[0]*srate):int(time_trim[1]*srate)]
clean_eeg, artifact_eeg = art.remove_eyeblinks(raw_trim, srate, n_clusters=10, fd_threshold=1.6)


## Plot cleaned EEG

In [27]:
chans_plot = range(3)   # List of integers with channels to plot

fig, ax = plt.subplots(len(chans_plot),1)
for chan in chans_plot:
    ax[chan].plot(time_vect, 1e6*(raw_trim[chan,:]-np.mean(raw_trim[chan,:])))
    # ax[chan].plot(time_vect, 1e6*(raw_trim[chan,:]))
    ax[chan].plot(time_vect, 1e6*(clean_eeg[chan,:]))
    ax[chan].grid()

ax[0].legend(['Contaminated', 'Clean'], loc='upper right')
ax[1].set_ylabel('EEG [$\mu$V]')
ax[2].set_xlabel('Time [sec]')
plt.show()

## Plot Power Spectral Densities

In [29]:
chans_psd = [6,7]
win = np.ones(np.size(clean_eeg, axis=1))
f_raw , psd_raw = signal.welch(raw_trim[chans_psd,:], fs=srate, window=win)
f_clean, psd_clean = signal.welch(clean_eeg[chans_psd,:], fs=srate, window=win)

psd_xlims = [10, 30]
psd_ylims = [0, 1e-10]
# psd_ylims = [0, None]

fig, ax = plt.subplots(2,1)
ax[0].plot(f_raw, psd_raw.T)
ax[0].set_title('RAW')
ax[0].set_ylabel('PSD [$V^2/Hz$]')
ax[0].set_xlim((psd_xlims[0], psd_xlims[1]))
ax[0].set_ylim((psd_ylims[0], psd_ylims[1]))
ax[0].grid()

ax[1].plot(f_clean, psd_clean.T)
ax[1].set_title('Cleaned')
ax[1].set_ylabel('PSD [$V^2/Hz$]')
ax[1].set_xlabel('Frequency [Hz]')
ax[1].set_xlim((psd_xlims[0], psd_xlims[1]))
ax[1].set_ylim((psd_ylims[0], psd_ylims[1]))
ax[1].grid()

plt.tight_layout()